In [ ]:
from torch.utils.data import DataLoader
from resnet import ResNet
from sklearn.model_selection import train_test_split
import numpy as np
from custom_dataset import FinalDataset
import torch
from torch import nn

In [ ]:
file_path= "Dataset.hdf5"
dataset = FinalDataset(file_path)
indices = np.arange(len(dataset))
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)
train_dataset = FinalDataset(file_path, indices=train_indices)
val_dataset = FinalDataset(file_path, indices=val_indices)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset.indices)
    print(f"size: {size}")
    print(len(dataloader))
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss_item = loss.item()
            current = batch * len(X)
            print(f"loss: {loss_item:>7f} [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset.indices)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()  
            predicted = (pred > 0.5).float()
            correct += (predicted == y).type(torch.float).sum().item()
    # Average loss and accuracy
    test_loss /= num_batches
    accuracy = (100 * correct) / size
    print(f"Test Error: \n Accuracy: {accuracy:>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(f"Correct: {correct}, Test Loss: {test_loss}, Num Batches: {num_batches}, Size: {size}")


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(35).to(device))

model = ResNet(12,24)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

model = model.to(device)
epochs = 5

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer, device)
    test_loop(val_loader, model, loss_fn, device)
    
print("Done!")
torch.save(model.state_dict(), "model.pth")